In [1]:
import sys
import numpy as np

sys.path.append("../src")

from nat2324.problems.sumplete import Sumplete
from nat2324.algorithms.ga import BinaryGeneticAlgorithm

%load_ext autoreload
%autoreload 2

In [3]:
sumplete = Sumplete(3, evaluation_type="absolute")
ga = BinaryGeneticAlgorithm(sumplete.evaluate, p_m=0.01)

In [4]:
solution = ga.run(max_generations=1000, patience=100, return_score=True, return_duration=True, return_num_gens=True)

Current best 1.00000000:  11%|█         | 106/1000 [00:00<00:02, 393.47it/s]


In [6]:
solution[0]

array([1, 1, 1, 1, 1, 1, 0, 0, 1])

In [8]:
sumplete.show(solution[0])

+-------------------+
|     1     1     3 | 5
|     5     3     8 | 16
|                 7 | 7
+-------------------+
      6     4    18
